## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [48]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import pickle
%matplotlib qt
%matplotlib inline

'''
Camera class is used to:
    - calculate calibration parameters for the camera based on a set of calibration images
    - store calibration parameters for later use
    - read in calibration parameters to save on processing time
    - undistort an image using calibration parameters, for later processing
    - undistort a set of test images using calibration parameters, for demo purposes
'''
class Camera():
    def __init__(self):
        # camera calibration parameters
        self.mtx = []
        self.dist = []

    # calculate camera calibration parameters based on the calibration images
    def calculate_calibration_parameters(self):
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane.

        # Make a list of calibration images
        images = glob.glob('camera_cal/calibration*.jpg')

        # Step through the list and search for chessboard corners
        for fname in images:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img_size = (img.shape[1], img.shape[0])

            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

            # If found, add object points, image points
            if ret == True:
                objpoints.append(objp)
                imgpoints.append(corners)

                # Draw and display the corners
                img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
                cv2.imshow('img',img)
                cv2.waitKey(500)

        cv2.destroyAllWindows()

        if len(objpoints) & len(imgpoints):
            # Do camera calibration given object points and image points
            ret, self.mtx, self.dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

        # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
        dist_pickle = {}
        dist_pickle["mtx"] = self.mtx
        dist_pickle["dist"] = self.dist
        pickle.dump( dist_pickle, open( "camera_cal/camera_calibration_parameters.p", "wb" ) )

        print("Created new calibration parameters: mtx=", self.mtx, "; dist=", self.dist)


    # Read in the saved calibration parameters
    def read_calibration_parameters(self):
        calibration_file_path = "camera_cal/camera_calibration_parameters.p"
        if os.path.isfile(calibration_file_path):
            dist_pickle = pickle.load( open( calibration_file_path, "rb" ) )
            self.mtx = dist_pickle["mtx"]
            self.dist = dist_pickle["dist"]

            print("Loaded calibration parameters: mtx=", self.mtx, "; dist=", self.dist)


    # read saved calibration parameters or calculate based on the calibration images
    def initialize_calibration_parameters(self):
        # try to read calibration parameters saved before (speed up processing during development)
        self.read_calibration_parameters()

        # if saved camera calibration parameters were not found - try to calibrate camera
        if not (len(self.mtx) & len(self.dist)):
            self.calculate_calibration_parameters()


    # if camera calibration parameters are initiaized properly - undistort test images
    def undistort_test_images(self):
        if len(self.mtx) & len(self.dist):

            in_folder = "test_images/"
            out_folder = "output_images/"

            if not os.path.exists(out_folder):
                os.makedirs(out_folder)

            source_files = os.listdir(in_folder)

            # Read sample images, undistort and write to the output folder
            for source_file in source_files:

                source_path = in_folder + source_file
                img = cv2.imread(source_path)

                undistorted = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)

                result_path = out_folder + source_file
                print("result_path=", result_path);
                cv2.imwrite(result_path, undistorted)

                # Draw and display undistorted image
                f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
                f.tight_layout()
                ax1.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image', fontsize=50)
                ax2.imshow(cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB))
                ax2.set_title('Undistorted Image', fontsize=50)
                plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
                plt.show()

    # undistort image using the camera calibration parameters
    def undistort_image(self, image):
        if len(self.mtx) & len(self.dist):
            return cv2.undistort(image, self.mtx, self.dist, None, self.mtx)
                



## And so on and so forth...

In [78]:
def debug_show_images(img1, title1, img2, title2, cmap1=None, cmap2=None):
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1, cmap=cmap1)
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(img2, cmap=cmap2)
    ax2.set_title(title2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()


In [7]:
class BinaryImage():
    def __init__(self):
        pass

In [79]:
# Calculate directional gradient and apply threshold
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    
    # grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # derivative
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # absolute value
    abs_sobel = np.absolute(sobel)
    
    # scale to 0-255
    abs_sobel_max = np.max(abs_sobel)
    abs_sobel = np.uint8(255*abs_sobel/abs_sobel_max)
    
    # apply threshold
    binary_output = np.zeros_like(abs_sobel)
    binary_output[(abs_sobel >= thresh[0]) & (abs_sobel <= thresh[1])] = 1
    
    # display the result
    #debug_show_images(cv2.cvtColor(image,cv2.COLOR_BGR2RGB), 'Original Image', binary_output, 'Thresholded Gradient', cmap2 = 'gray')
    
    return binary_output

# Calculate gradient magnitude and apply threshold
def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):

    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    
    # grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # derivative
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # magnitude
    sobel = np.zeros_like(sobel_x)
    sobel = (sobel_x**2 + sobel_y**2)**(1/2.0)
    
    # scale to 0-255
    sobel_max = np.max(sobel)
    sobel = np.uint8(255*sobel/sobel_max)
    
    # apply threshold
    binary_output = np.zeros_like(sobel)
    binary_output[(sobel >= mag_thresh[0]) & (sobel <= mag_thresh[1])] = 1

    # display the result
    #debug_show_images(cv2.cvtColor(image,cv2.COLOR_BGR2RGB), 'Original Image', binary_output, 'Thresholded Magnitude', cmap2 = 'gray')
    
    return binary_output

# Calculate gradient direction and apply threshold
def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    
    # grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # derivative
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # absolute value and direction of the gradient
    grad_direction = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    
    # apply threshold
    binary_output = np.zeros_like(grad_direction)
    binary_output[(grad_direction >= thresh[0]) & (grad_direction <= thresh[1])] = 1
    
    # display the result
    #debug_show_images(cv2.cvtColor(image,cv2.COLOR_BGR2RGB), 'Original Image', binary_output, 'Thresholded Grad. Dir.', cmap2 = 'gray')
    
    return binary_output


# calculate binary image based on different gradient thresholds (absolute, magnitude, direction)
def gradient_threshold(image):
    
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(20, 100))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=3, thresh=(20, 100))
    mag_binary = mag_thresh(image, sobel_kernel=9, mag_thresh=(30, 100))
    dir_binary = dir_threshold(image, sobel_kernel=15, thresh=(0.7, 1.3))

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # display the result
    #debug_show_images(cv2.cvtColor(image,cv2.COLOR_BGR2RGB), 'Original Image', combined, 'Combined Grad.-Thresh. Image', cmap2 = 'gray')
    
    return combined


In [80]:
def colour_threshold(image, s_thresh=(170, 255), sx_thresh=(20, 100)):
    #image = np.copy(image)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x on luminance channel
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary
    
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return color_binary


In [85]:
def combined_threshold(gradient_binary, colour_binary):
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(gradient_binary)
    combined_binary[(gradient_binary == 1) | (colour_binary == 1)] = 1
    return combined_binary


In [114]:
def perspective_transform(image):
    
    # calculate region of interest
    ysize = image.shape[0]
    xsize = image.shape[1]

    near_distance = ysize-1
    far_distance = ysize*0.645
    mid_x = (xsize-1)/2

    left_bottom = [0+150, near_distance]
    right_bottom = [xsize-1-150, near_distance]

    left_apex = [mid_x-75, far_distance]
    right_apex = [mid_x+75, far_distance]

    img_size = (image.shape[1], image.shape[0])
    src = np.float32(
        [left_bottom,
        left_apex,
        right_apex,
        right_bottom])
    dst = np.float32(
        [[0, ysize-1],
        [0, 0],
        [xsize-1, 0],
        [xsize-1, ysize-1]])
    
    # draw source region
    for i in range(0, len(src)-1):
        cv2.line(image, (src[i][0], src[i][1]), (src[i+1][0], src[i+1][1]), [255, 0, 0], 2)
    
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(image, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped


In [ ]:
camera = Camera()
camera.initialize_calibration_parameters()

in_folder = "test_images/"
out_folder = "output_images/"

if not os.path.exists(out_folder):
    os.makedirs(out_folder)

source_files = os.listdir(in_folder)

# Read and process sample images
for source_file in source_files:

    # read test image in
    source_path = in_folder + source_file
    img = cv2.imread(source_path)

    # undistort according to the camera calibration parameters
    undistorted = camera.undistort_image(img)
    debug_show_images(cv2.cvtColor(img,cv2.COLOR_BGR2RGB), 'Original Image', cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB), 'Undistorted Image')

    # write undostorted image to output folder, if needed
    result_path = out_folder + 'undistorted_' + source_file
    print("Writing to result_path=", result_path);
    cv2.imwrite(result_path, undistorted)
    
    # find threholded binary image using gradients
    thresholded_grad = gradient_threshold(undistorted)
    #debug_show_images(cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB), 'Undistorted Image', thresholded_grad, 'Thresholded by Gradients', cmap2 = 'gray')
    
    # another try with colour transformation and gradient
    thresholded_col = colour_threshold(undistorted)
    #debug_show_images(cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB), 'Undistorted Image', thresholded_col, 'Thresholded by Colour', cmap2 = 'gray')
    
    # combine into a single threholded binary image
    combined_binary = combined_threshold(thresholded_grad, thresholded_col)
    debug_show_images(cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB), 'Undistorted Image', combined_binary, 'Thresholded Image', cmap2 = 'gray')

    # write threholded image to output folder, if needed
    result_path = out_folder + 'threholded_' + source_file
    print("Writing to result_path=", result_path);
    cv2.imwrite(result_path, combined_binary * 255)
    
    warped_image = perspective_transform(undistorted)
    debug_show_images(cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB), 'Undistorted Image', cv2.cvtColor(warped_image,cv2.COLOR_BGR2RGB), 'Warped Image')

    #break